## Basic things

In [1]:
%%capture
# scipy is to solve some warning
!pip install --upgrade datasets transformers scipy

In [2]:
import datasets 
import transformers
print(transformers.__version__) # some older vision cause bugs, I use 4.34.0
print(datasets.__version__) # some older version cause bugs, I use 2.14.5

4.34.0
2.14.5


In [3]:
!rm -r output_control/ #if older stuff exists
!git clone https://github.com/TeunvdWeij/output_control 
!cd output_control

rm: cannot remove 'output_control/': No such file or directory
Cloning into 'output_control'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 227 (delta 113), reused 166 (delta 54), pack-reused 0
Receiving objects: 100% (227/227), 140.10 KiB | 2.06 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [5]:
import os
os.chdir('/kaggle/working/output_control')
os.getcwd()

'/kaggle/working/output_control'

## Load model

In [8]:
%load_ext autoreload
%autoreload 2
import torch
from tqdm import tqdm

from src.model import Llama2Helper
from src.utils import load_pile, get_subset_from_dataset

# put your own hugging face token here
hf_token = ""

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
dataset = load_pile(mode="only_code", batch_size=1, split="train", iterable=True)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [10]:
model_name = "meta-llama/Llama-2-7b-hf"
model = Llama2Helper(model_name=model_name, hf_token=hf_token)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [11]:
layer = 29

In [52]:
avg_acts = 0
total_samples = 1000
for i, code_sample in tqdm(enumerate(dataset, 1), total=total_samples):    
    encoded = model.tokenizer.encode(
        code_sample['text'], 
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=4096)
    model.get_logits(encoded)
    
    acts = model.get_last_activations(layer)[:, -1, :]
    avg_acts = (avg_acts * (i-1) + acts) / (i+1)
        
    if i >= total_samples:
        break
        
avg_acts.shape, torch.max(avg_acts)


100%|█████████▉| 999/1000 [27:53<00:01,  1.67s/it]


(torch.Size([1, 4096]), tensor(13.0234, device='cuda:1', dtype=torch.float16))

In [53]:
torch.save(avg_acts, f"acts_v0.7_{model_name[-13:]}_{total_samples}.pt")

In [54]:
torch.max(avg_acts)

tensor(13.0234, device='cuda:1', dtype=torch.float16)

In [58]:

# check for infs and nans, see https://stackoverflow.com/questions/48158017/pytorch-operation-to-detect-nans
(avg_acts == torch.inf).any(), (avg_acts != avg_acts).any() 

(tensor(False, device='cuda:1'), tensor(False, device='cuda:1'))